# RK4 Model

### Trajectory of golf ball 

In this notebook, we compare the trajectory of a golf ball computed with a model that includes drag force and Magnus effect to the the trajectory given by a HD Golf simulator. Our model uses the same initial conditions as the HD Golf simulator.

## Resources

Note that it's not unusual to see the variable $y$ used for $v_x$. This can be confusing.

- [Coupled masses](http://scipy-cookbook.readthedocs.io/items/CoupledSpringMassSystem.html). This is a clear example.
- [Projectile motion with drag](http://ipython-books.github.io/123-simulating-an-ordinary-differential-equation-with-scipy/). This uses some interesting numpy tricks like the `r_()` function, so it's harder to figure out.

In [1]:
import numpy as np #math and arrays
import matplotlib.pyplot as plt #used for graphing
import scipy.integrate as spi 
import pandas as pd

In [2]:
# Read data file and define dataframe object (df for dataframe)
df = pd.read_table('TESTDATA.txt')
df.head(13)

,Shot Distance (m),Carry (m),Bounce & Roll (m),Ball Speed (m/s),Club Speed (m/s),Launch Angle (deg),Club Path,Club Face Angle (deg),Smash Factor,Horizontal Angle (deg),Apex (m),Impact Position,Spin (rpm),X Peak Point (m),Y Peak Point (m),X End Point (m),Y End Point (m)
0,117.9,116.7,1.1,43.0,38.3,21.8,-2.3,0.0,1.12,-2.3,20.9,TOE,9746,71,20.9,117,0
1,111.6,110.9,0.7,41.4,35.7,25.9,-2.2,0.0,1.16,-2.2,23.1,TOE,9566,67,23.1,111,0
2,112.4,122.1,0.3,45.3,38.5,25.4,-2.8,0.0,1.18,-2.8,27.5,CENTER,9898,75,27.5,122,0
3,117.7,115.4,2.3,41.5,37.1,26.2,-0.6,0.0,1.12,-0.6,23.4,TOE,7865,69,23.4,117,0
4,117.9,115.6,2.4,42.0,35.9,22.0,-2.2,0.1,1.17,-2.3,19.7,HEEL,8613,70,19.7,118,0
5,128.6,127.0,1.7,45.1,37.5,26.3,-0.4,0.2,1.20,-0.2,28.1,CENTER,7895,77,28.1,128,0
6,124.4,123.1,1.2,44.8,38.7,28.1,-2.3,0.2,1.16,-2.5,29.6,CENTER,8452,75,29.6,124,0
7,118.8,118.1,0.6,43.6,36.1,24.7,-3.2,0.2,1.21,-3.4,24.7,CENTER,9651,72,24.7,118,0
8,104.4,102.1,2.3,37.6,34.9,28.2,-2.0,0.2,1.08,-1.8,20.7,TOE,7616,60,20.7,104,0
9,117.7,116.3,1.5,42.3,35.7,26.7,1.1,0.3,1.18,0.9+,25.0,CENTER,8486,70,25.0,117,0


In [3]:
rows=df['Carry (m)'].size
rows

12

In [4]:
def f(r, t, p):    
    """
    Defines the differential equations for the system.

    Arguments:
        r :  array of the state variables:
                  r = [x,vx,y,vy]
        t :  array of time values
        p :  array of the parameters:
                  p = [m,g,rho,Cd,Cm,R,A,spinrate]
    """
    m=p[0]
    g=p[1]
    rho=p[2]
    Cd=p[3]
    Cm=p[4]
    R=p[5]
    A=p[6]
    spinrate=p[7]
    
    x=r[0]
    y=r[2]
    vx=r[1]
    vy=r[3]

    # compute the derivative of velocity 
    speed=np.sqrt(vx**2+vy**2)
    vxdot = -1/2*rho*Cd*A/m*speed*vx - 1/2*Cm*rho*np.pi*R**3*spinrate*vy/m
    vydot = -g + -1/2*rho*Cd*A/m*speed*vy + 1/2*Cm*rho*np.pi*R**3*spinrate*vx/m
    
    #return [vx,vy,vxdot,vydot]
    return np.array([vx,vxdot,vy,vydot])

In [5]:
def testCoef(Cd, Cm):
    #Return RMS 
    #constants for the model
    rho = 1.225 #air density, kg/m^3
    R = 0.04267/2 #radius of ball, m
    A = np.pi*R**2 #area
    g = 9.8        #Earth's grav field strength
    m = 0.0456     #mass of the ball, kg
    spinrate = 0   #this depends on the shot and will be changed within the main loop

    #pass parameters to the ode function
    params=[m,g,rho,Cd,Cm,R,A,spinrate]
    
    #loop through all rows
    rms_total = 0
    for row in range(0,rows):    

        df_row=df.iloc[row]

        #data for HD Golf trajectory
        x_end_HD = df_row['Carry (m)']
        x_apex_HD = df_row['X Peak Point (m) ']
        y_apex_HD = df_row['Y Peak Point (m)']

        #Do not need a Y_end_HD because the simulator has the golf ball ending at y=0

        x_hdgolf = [0, x_apex_HD, x_end_HD]
        y_hdgolf = [0, y_apex_HD, 0]    

        #initial conditions for the ball
        launch_angle = df_row['Launch Angle (deg)'] * np.pi/180 #radians
        launch_speed = df_row['Ball Speed (m/s)'] #m/s
        rpm = df_row['Spin (rpm)'] #rev per minute 
        spinrate = rpm /60 *2* np.pi
        params[7]=spinrate #this variable is kept in the parameters list

        # trajectory data array: [x,vx,y,vy]
        r0 = np.zeros(4) #four columns of data

        #Define initial positions and velocity
        r0[0] = 0 # x(0)
        r0[1] = launch_speed*np.cos(launch_angle) # vx(0)
        r0[2] = 0 # y(0)
        r0[3] = launch_speed*np.sin(launch_angle) # vy(0)

        #Solve the DE using RK4
        t_array = np.linspace(0., 7, 1000)
        r = spi.odeint(f, r0, t_array,args=(params,))


        #Print this line to differentiate between trials 
        #print("\n\n")

        y_apex_model=np.max(r[:,2]) #y value of peak
        apex_index, = np.where(r[:,2] == y_apex_model)
        x_apex_model = r[:,0][int(apex_index)]
        #print("model apex (x,y) = (", x_apex_model, ", ", y_apex_model, ")m")
        #print("HD apex (x,y) = (", x_apex_HD, ", ", y_apex_HD, ")m")

        y_no_initial=np.delete(r[:,2], 0) #array without the initial values
        abs_y=np.abs(y_no_initial) # |y| so negative values will be positive
        index_ground=np.argmin(abs_y) #index of array when |y| is a min (at ground)
        carry=r[:,0][index_ground+1] #x value when projectile hits the ground
        x_end_model=carry
        y_end_model=r[:,2][index_ground+1] #y value when projectile hits the ground
        #print("model end (x,y) = (", x_end_model, ", ", y_end_model, ")m")
        #print("HD end (x,y) = (", x_end_HD, ", 0)m")




        #Difference of peak point between HD and model 
        dPeak_sq = (x_apex_HD - x_apex_model)**2 + (y_apex_HD - y_apex_model)**2

        #Difference of end point between HD and model 
        dEnd_sq = (x_end_HD - x_end_model)**2 + (y_end_model)**2 

        rms = np.sqrt((dPeak_sq + dEnd_sq)/2)
        rms_total = rms_total + rms 

        #print(rms)

    #print("\n\n\n\n")        

    RMS_avg = rms_total/rows
    return RMS_avg

    #print('Average RMS:', RMS_avg)

In [26]:
#optimization 
for cd in np.arange(0.3,0.5,0.01): 
    Average_RMS = testCoef(cd,0.56)
    print(cd, Average_RMS)
    
print ("\n\n")

for cm in np.arange(0.45,0.65,0.01):
    Average_RMS = testCoef(0.39,cm)
    print(cm, Average_RMS)

0.3 12.9565159128
0.31 11.3746354426
0.32 9.87341294318
0.33 8.43951986896
0.34 7.08815975374
0.35 5.87788086616
0.36 4.78125664719
0.37 3.86943714511
0.38 3.31758938887
0.39 3.09425187176
0.4 2.99876620694
0.41 3.16796232953
0.42 3.62910411674
0.43 4.27082059363
0.44 5.10571801352
0.45 6.07334641357
0.46 6.99173437051
0.47 7.93783887073
0.48 8.84600834364
0.49 9.74618685463



0.45 5.34097367503
0.46 4.95018576979
0.47 4.58464348361
0.48 4.25895221206
0.49 3.95510737604
0.5 3.69177986591
0.51 3.45071560155
0.52 3.27434269369
0.53 3.14828349378
0.54 3.09322894491
0.55 3.0838182
0.56 3.09425187176
0.57 3.14322817338
0.58 3.19507915184
0.59 3.29110021399
0.6 3.40946749164
0.61 3.60036990462
0.62 3.83786412218
0.63 4.10642388608
0.64 4.40036046197
